In [1]:
from itertools import product
import numpy as np
import networkx as nx
import time

from adapt import adapt
from callbacks import MaxGradientStopper, ParameterStopper, EnergyErrorPrinter
from simulator import make_connectivity_pool, plus_state
from scipy.optimize import minimize

In [2]:
num_qubits = 3
ham = np.diag(np.random.uniform(size=2**num_qubits))

In [3]:
ham

array([[0.71432954, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.43249917, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.9351176 , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.07356463, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.6280961 ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.45035486, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70418487, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.32046303]])

In [4]:
#pool = make_connectivity_pool(nx.path_graph(num_qubits))

In [5]:
pool = list(product(range(4), repeat=num_qubits))

In [6]:
def optimizer(*args, **kwargs):
    return minimize(*args, method='Nelder-Mead', **kwargs)

In [12]:
callbacks = [
    MaxGradientStopper(1e-3),
    ParameterStopper(50),
    EnergyErrorPrinter(np.min(np.linalg.eigvalsh(ham)))
]

In [13]:
t_i = time.time()

result = adapt(
    ham=ham,
    pool=pool,
    optimizer=optimizer,
    callbacks=callbacks,
    initial_state=plus_state(num_qubits),
    new_parameter=1e-2
)

t_f = time.time()

print(f'Time elapsed: {np.floor(t_f-t_i)}s')


Energy error at step 2 reported: 0.36054869791875366
Log_10(|Energy error|) at step 2 reported: -0.4430360685061119

Energy error at step 3 reported: 0.3203824582350242
Log_10(|Energy error|) at step 3 reported: -0.4943312706785883
Improvement from last (positive good): 0.040166239683729454
Log_10(|improvement|): -1.3961388245055169

Energy error at step 4 reported: 0.3203824582676876
Log_10(|Energy error|) at step 4 reported: -0.4943312706343114
Improvement from last (positive good): -3.266337200713565e-11
Log_10(|improvement|): -10.485938982841901

Energy error at step 5 reported: 0.12344920219668121
Log_10(|Energy error|) at step 5 reported: -0.9085117123962739
Improvement from last (positive good): 0.19693325607100637
Log_10(|improvement|): -0.7056809384653744

Energy error at step 6 reported: 3.0720439635062435e-06
Log_10(|Energy error|) at step 6 reported: -5.512572573480517
Improvement from last (positive good): 0.1234461301527177
Log_10(|improvement|): -0.9085225199861048
Time

In [14]:
result.dataframe

,energy,gradients,opt_params,min_vector,max_grad,op_indices
0,0.532326,"[0.0, 0.0, -0.4262116082893272, 0.0, 0.0, 0.0,...",[],None,[0.4262116082893272],[]
1,0.434113,"[0.0, 0.0, -1.9745017750867218e-05, 0.0, 0.0, ...",[-0.7853750000000022],None,[0.4741989942207954],[14]
2,0.393947,"[0.0, 0.0, 2.7419413668872166e-10, 0.0, 0.0, 0...","[-0.7853739271852374, -0.7854047994245659]",None,[0.6407649160056544],"[14, 47]"
3,0.393947,"[0.0, 0.0, 9.598482435792955e-10, 0.0, 0.0, 0....","[-0.7953964807919882, -0.7854324560199057, 0.0...",None,[0.6407649157515533],"[14, 47, 30]"
4,0.197014,"[0.0, 0.0, -1.016040222245962e-09, 0.0, 0.0, 0...","[-0.42359383478166845, 4.3377196928369765e-05,...",None,[0.24689840038641744],"[14, 47, 30, 22]"
5,0.073568,"[0.0, 0.0, 1.3771103499914214e-09, 0.0, 0.0, 0...","[-0.6166524388212524, 0.002353416604436529, -0...",None,[0.002610077475154464],"[14, 47, 30, 22, 32]"
6,0.073565,"[0.0, 0.0, -3.7395321034079756e-12, 0.0, 0.0, ...","[-0.5670021903574958, 0.0028416452504930246, -...",None,[1.2267306652317447e-05],"[14, 47, 30, 22, 32, 37]"


In [15]:
result.halt_reason

'Gradient threshhold satisfied'

In [16]:
result.step_history[-1]['energy'] - np.min(np.linalg.eigvalsh(ham))

8.384060112831548e-11